In [1]:
# Initial imports# Initi 
import numpy as np
import pandas as pd 
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
%matplotlib inline

import random
import requests
from bs4 import BeautifulSoup
import warnings
warnings.filterwarnings('ignore')

In [2]:
base_url  = "https://sofifa.com/players?offset="
offset = 0
columns = ['ID', 'Name', 'Age', 'Photo', 'Nationality', 'Flag', 'Overall', 'Potential', 'Club', 
           'Club Logo', 'Value', 'Wage', 'Special', 'Position']
data = DataFrame(columns=columns)
for offset in range(225):
    url = base_url + str(offset*80)
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    table_body = soup.find('tbody')
    counter = 0
    for row in table_body.findAll('tr'):
        td = row.findAll('td')
        picture = td[0].find('img').get('data-src')
        pid = td[0].find('img').get('id')
        pos = '|'.join([p.text for p in td[1].findAll('span')])
        nationality = td[1].find('a').get('title')
        flag_img = td[1].find('img').get('data-src')
        name = td[1].findAll('a')[1].text
        age = td[2].text.strip()
        overall = td[3].text.strip()
        potential = td[4].text.strip()
        club = td[5].find('a').text
        club_logo = td[5].find('img').get('data-src')
        value = td[7].text
        wage = td[8].text
        special = td[10].find('span').text.strip()
        player_data = DataFrame([[pid, name, age, picture, nationality, flag_img, overall, 
                                  potential, club, club_logo, value, wage, special, pos]])
        player_data.columns = columns
        data = data.append(player_data, ignore_index=True)
        counter+=1
    offset+=1
    if offset % 10:
      print(offset)
    data.to_csv('data/full_player_data.csv', encoding='utf-8')

1
2
3
4
5
6
7
8
9
11
12
13
14
15
16
17
18
19
21
22
23
24
25
26
27
28
29
31
32
33
34
35
36
37
38
39
41
42
43
44
45
46
47
48
49
51
52
53
54
55
56
57
58
59
61
62
63
64
65
66
67
68
69
71
72
73
74
75
76
77
78
79
81
82
83
84
85
86
87
88
89
91
92
93
94
95
96
97
98
99
101
102
103
104
105
106
107
108
109
111
112
113
114
115
116
117
118
119
121
122
123
124
125
126
127
128
129
131
132
133
134
135
136
137
138
139
141
142
143
144
145
146
147
148
149
151
152
153
154
155
156
157
158
159
161
162
163
164
165
166
167
168
169
171
172
173
174
175
176
177
178
179
181
182
183
184
185
186
187
188
189
191
192
193
194
195
196
197
198
199
201
202
203
204
205
206
207
208
209
211
212
213
214
215
216
217
218
219
221
222
223
224
225


In [2]:
data = pd.read_csv('data/full_player_data.csv')

In [3]:
data.head(3)

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Position
0,0,239837,A. Mac Allister,19,https://cdn.sofifa.org/players/4/19/239837.png,Argentina,https://cdn.sofifa.org/flags/52.png,72,86,Argentinos Juniors,https://cdn.sofifa.org/teams/2/light/111019.png,€6.5M,€15K,1958,CM|CAM|On Loan
1,1,186832,K. Asamoah,29,https://cdn.sofifa.org/players/4/19/186832.png,Ghana,https://cdn.sofifa.org/flags/117.png,79,79,Inter,https://cdn.sofifa.org/teams/2/light/44.png,€9.5M,€62K,2167,LB
2,2,211110,P. Dybala,24,https://cdn.sofifa.org/players/4/19/211110.png,Argentina,https://cdn.sofifa.org/flags/52.png,89,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€89M,€205K,2092,CAM|RW


In [10]:
data.to_csv('data/basicplayerdata.csv', encoding='utf-8')

In [21]:
player_data_url  = 'https://sofifa.com/player/'
r = 0
for index, row in data.iterrows():
    skill_names = []
    skill_map = {'ID' : str(row['ID'])}
    url = player_data_url + str(row['ID'])
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    categories = soup.findAll('div', {'class': 'col-4'})
    for category in categories[:-1]:
        skills = category.findAll('li')
        for skill in skills:
            a = skill.text.split()
            a.reverse()
            value = a.pop()
            a.reverse()
            n = ' '.join(a)
            skill_names.append(n)
            skill_map[str(n)] = value
    master_data = DataFrame(columns=skill_names)
    break

In [ ]:
player_data_url = 'https://sofifa.com/player/'
r = 0
for index, row in data.iterrows():
    skill_names = []
    skill_map = {'ID' : str(row['ID'])}
    url = player_data_url + str(row['ID'])
    source_code = requests.get(url)
    plain_text = source_code.text
    soup = BeautifulSoup(plain_text)
    categories = soup.findAll('div', {'class': 'col-4'})
    for category in categories[:-1]:
        skills = category.findAll('li')
        for skill in skills:
            a = skill.text.split()
            a.reverse()
            value = a.pop()
            a.reverse()
            n = ' '.join(a)
            skill_names.append(n)
            skill_map[str(n)] = value
    attr_data = DataFrame(columns=skill_names)
    for key in skill_map.keys():
        attr_data.loc[r,key] = skill_map[key]
    r = r + 1
    master_data = pd.concat([master_data, attr_data])
    if r % 100 == 0:
        print(r)
        master_data.to_csv('data/PlayerAttributeData.csv', encoding='utf-8', mode='a')

100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400


In [5]:
master_data = pd.read_csv('data/PlayerAttributeData.csv')
master_data.head(3)

,Unnamed: 0,Acceleration,Aggression,Agility,Balance,Ball Control,Composure,Crossing,Curve,Dribbling,...,Reactions,Short Passing,Shot Power,Sliding Tackle,Sprint Speed,Stamina,Standing Tackle,Strength,Vision,Volleys
0,0.0,72,51,69,71,75,72,72,73,73,...,69,75,74,51,71,69,54,61,76,63
1,1.0,77,82,85,83,81,76,78,72,82,...,78,81,84,80,77,76,79,75,76,76
2,2.0,87,48,91,85,92,84,82,88,92,...,86,87,82,20,83,80,20,65,87,88


In [6]:
data.head(3)

,Unnamed: 0,ID,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,Wage,Special,Position
0,0,239837,A. Mac Allister,19,https://cdn.sofifa.org/players/4/19/239837.png,Argentina,https://cdn.sofifa.org/flags/52.png,72,86,Argentinos Juniors,https://cdn.sofifa.org/teams/2/light/111019.png,€6.5M,€15K,1958,CM|CAM|On Loan
1,1,186832,K. Asamoah,29,https://cdn.sofifa.org/players/4/19/186832.png,Ghana,https://cdn.sofifa.org/flags/117.png,79,79,Inter,https://cdn.sofifa.org/teams/2/light/44.png,€9.5M,€62K,2167,LB
2,2,211110,P. Dybala,24,https://cdn.sofifa.org/players/4/19/211110.png,Argentina,https://cdn.sofifa.org/flags/52.png,89,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€89M,€205K,2092,CAM|RW


In [7]:
full_data = pd.merge(data, master_data, left_index=True, right_index=True)

In [8]:
full_data.to_csv('data/Allplayer.csv', encoding='utf-8')

In [9]:
master_data.to_csv('data/PlayerAttributeData.csv', encoding='utf-8')

In [10]:
full_data.to_csv('data/Dataset.csv', encoding='utf-8')

In [11]:
full_data.head(1).T

,0
Unnamed: 0_x,0
ID_x,239837
Name,A. Mac Allister
Age,19
Photo,https://cdn.sofifa.org/players/4/19/239837.png
Nationality,Argentina
Flag,https://cdn.sofifa.org/flags/52.png
Overall,72
Potential,86
Club,Argentinos Juniors


In [14]:
full_data.drop('Unnamed: 0_x', 1,  inplace=True)

In [15]:
full_data.head(3)

,ID_x,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,...,Reactions,Short Passing,Shot Power,Sliding Tackle,Sprint Speed,Stamina,Standing Tackle,Strength,Vision,Volleys
0,239837,A. Mac Allister,19,https://cdn.sofifa.org/players/4/19/239837.png,Argentina,https://cdn.sofifa.org/flags/52.png,72,86,Argentinos Juniors,https://cdn.sofifa.org/teams/2/light/111019.png,...,69,75,74,51,71,69,54,61,76,63
1,186832,K. Asamoah,29,https://cdn.sofifa.org/players/4/19/186832.png,Ghana,https://cdn.sofifa.org/flags/117.png,79,79,Inter,https://cdn.sofifa.org/teams/2/light/44.png,...,78,81,84,80,77,76,79,75,76,76
2,211110,P. Dybala,24,https://cdn.sofifa.org/players/4/19/211110.png,Argentina,https://cdn.sofifa.org/flags/52.png,89,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,...,86,87,82,20,83,80,20,65,87,88


In [16]:
full_data.drop('ID_x', 1,  inplace=True)

In [17]:
full_data['ID_y'].head(3)

0    239837
1    186832
2    211110
Name: ID_y, dtype: object

In [18]:
f = full_data.rename(index=str, columns={"ID_y": "ID"})

In [19]:
f['ID'].head(3)

0    239837
1    186832
2    211110
Name: ID, dtype: object

In [20]:
f.to_csv('data/Dataset.csv', encoding='utf-8')

In [21]:
f.head(3)

,Name,Age,Photo,Nationality,Flag,Overall,Potential,Club,Club Logo,Value,...,Reactions,Short Passing,Shot Power,Sliding Tackle,Sprint Speed,Stamina,Standing Tackle,Strength,Vision,Volleys
0,A. Mac Allister,19,https://cdn.sofifa.org/players/4/19/239837.png,Argentina,https://cdn.sofifa.org/flags/52.png,72,86,Argentinos Juniors,https://cdn.sofifa.org/teams/2/light/111019.png,€6.5M,...,69,75,74,51,71,69,54,61,76,63
1,K. Asamoah,29,https://cdn.sofifa.org/players/4/19/186832.png,Ghana,https://cdn.sofifa.org/flags/117.png,79,79,Inter,https://cdn.sofifa.org/teams/2/light/44.png,€9.5M,...,78,81,84,80,77,76,79,75,76,76
2,P. Dybala,24,https://cdn.sofifa.org/players/4/19/211110.png,Argentina,https://cdn.sofifa.org/flags/52.png,89,94,Juventus,https://cdn.sofifa.org/teams/2/light/45.png,€89M,...,86,87,82,20,83,80,20,65,87,88
